# GP3 Trading Model

In [ ]:
from gp3.config_private import ALPACA_API_KEY, ALPACA_API_SECRET
API_BASE_URL = 'https://paper-api.alpaca.markets'

In [ ]:
import sys
import os
import pickle
import json
import shutil
import time
import random
sys.path.append('gp3')
from gp3.config import INDICATORS, CDL, TRAIN_START_DATE, TRAIN_END_DATE
from gp3.config import TEST_START_DATE, TEST_END_DATE
from gp3.config_tickers import GP3_TICKERS
from gp3.data_processor import DataProcessor

from gp3.elegantRL.env.StockTradingEnv import StockTradingEnv
from gp3.elegantRL.agent.AgentPPO import AgentPPO
from gp3.elegantRL.agent.DRLAgent import DRLAgent



In [ ]:
print(GP3_TICKERS)

In [ ]:
print(INDICATORS)

In [ ]:
env = StockTradingEnv

## Training

In [ ]:
def train(
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    **kwargs,
):
    # Create 'train_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "train_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'train_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array = dp.df_to_array(data, if_vix)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array), f)

    # read parameters
    cwd = kwargs.get("cwd", "./" + str(model_name))
    
    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        break_step = kwargs.get("break_step", 1e6)
        erl_params = kwargs.get("erl_params")
        agent = DRLAgent_erl(
            env=env,
            price_array=price_array,
            tech_array=tech_array,
            turbulence_array=turbulence_array,
        )
        model = agent.get_model(model_name, model_kwargs=erl_params)
        agent.train_model(
            model=model, cwd=cwd, total_timesteps=break_step
        )

def test(
    start_date,
    end_date,
    ticker_list,
    data_source,
    time_interval,
    technical_indicator_list,
    drl_lib,
    env,
    model_name,
    if_vix=True,
    **kwargs,
):
    # Create 'test_data' folder in the current working directory if it doesn't exist
    folder_path = os.path.join(os.getcwd(), "test_data")
    os.makedirs(folder_path, exist_ok=True)

    # Set the file paths within the 'test_data' folder
    data_file = os.path.join(folder_path, f"data_{start_date}_{end_date}.pkl")
    arrays_file = os.path.join(folder_path, f"arrays_{start_date}_{end_date}.pkl")

    if os.path.exists(data_file) and os.path.exists(arrays_file):
        # Load the saved data
        with open(data_file, "rb") as f:
            data = pickle.load(f)
        with open(arrays_file, "rb") as f:
            price_array, tech_array, turbulence_array = pickle.load(f)
    
    else:
        # download data
        dp = DataProcessor(data_source, **kwargs)
        data = dp.download_data(ticker_list, start_date, end_date, time_interval)
        data = dp.clean_data(data)
        data = dp.add_technical_indicator(data, technical_indicator_list)
        if if_vix:
            data = dp.add_vix(data)
        else:
            data = dp.add_turbulence(data)
        price_array, tech_array, turbulence_array = dp.df_to_array(data, if_vix)
        
        # Save the data and arrays
        with open(data_file, "wb") as f:
            pickle.dump(data, f)
        with open(arrays_file, "wb") as f:
            pickle.dump((price_array, tech_array, turbulence_array), f)
    env_config = {
        "price_array": price_array,
        "tech_array": tech_array,
        "turbulence_array": turbulence_array,
        "if_train": False,
    }
    env_instance = env(config=env_config)

    cwd = kwargs.get("cwd", "./" + str(model_name))

    if drl_lib == "elegantrl":
        DRLAgent_erl = DRLAgent
        episode_total_assets, max_drawdown, avg_reward  = DRLAgent_erl.DRL_prediction(
            env=env_instance,
            cwd=cwd,
        )
        
        return episode_total_assets, max_drawdown, avg_reward

def update_leaderboard(cwd, leaderboard_dir, episode_total_assets, max_drawdown, avg_reward, erl_params):
    actor_folder_name = f"actor_{round(episode_total_assets)}"
    leaderboard_path = os.path.join(leaderboard_dir, actor_folder_name)

    # Move and rename the directory
    if not os.path.exists(leaderboard_path):
        shutil.move(cwd, leaderboard_path)
    else:
        shutil.rmtree(cwd, ignore_errors=True)
    
    time.sleep(0.02)
        
    leaderboard_data_path = os.path.join(leaderboard_dir, 'leaderboard_data.json')

    # Load the leaderboard data if it exists
    if os.path.isfile(leaderboard_data_path):
        with open(leaderboard_data_path, 'r') as f:
            leaderboard_data = json.load(f)
    else:
        leaderboard_data = []

    # Add the new trial data
    new_entry = {
        'episode_total_assets': float(episode_total_assets),
        'max_drawdown': float(max_drawdown),
        'avg_reward': float(avg_reward),
        'hyperparameters': erl_params,
        'model_dir': leaderboard_path,
    }

    leaderboard_data.append(new_entry)

    # Sort the leaderboard data based on episode_total_assets
    leaderboard_data.sort(key=lambda x: x['episode_total_assets'], reverse=True)

    # Keep only the top 10 models
    leaderboard_data = leaderboard_data[:10]

    # Delete the models outside of the top 10
    best_models_dirs = {entry['model_dir'] for entry in leaderboard_data}
    for model_dir in os.listdir(leaderboard_dir):
        full_model_dir_path = os.path.join(leaderboard_dir, model_dir)
        if full_model_dir_path not in best_models_dirs and os.path.isdir(full_model_dir_path):
            shutil.rmtree(full_model_dir_path)
    time.sleep(0.02)
    # Save the updated leaderboard data
    with open(leaderboard_data_path, 'w') as f:
        json.dump(leaderboard_data, f, indent=4)

In [ ]:
run_time = (3600 * 1)

start_time = time.time()

while True:
    learning_rate = random.uniform(1e-6, 1e-3)
    weight_decay = random.uniform(1e-5, 1e-3)
    lambda_entropy = random.uniform(1e-2, 5e-2)
    lambda_gae_adv = round(random.uniform(0.5, 0.99), 2)
    clip_grad_norm = 3.0
    ratio_clip = 0.25
    batch_size = 256
    gamma = 0.995
    net_dimension = [256, 128, 1024]
    horizon_len = 1024
    repeat_times = 16
    optimizer = "RAdam"
    activation = "GELU"
    reward_scaling = 2 ** -11
    random_seed = 42

    erl_params = {
        "learning_rate": learning_rate,
        "weight_decay": weight_decay,
        "lambda_entropy": lambda_entropy,
        "batch_size": batch_size,
        "gamma": gamma,
        "net_dimension": net_dimension,
        "horizon_len": horizon_len,
        "repeat_times": repeat_times,
        "optimizer": optimizer,
        "clip_grad_norm": clip_grad_norm,
        "ratio_clip": ratio_clip,
        "lambda_gae_adv": lambda_gae_adv,
        "activation": activation,
        "reward_scaling": reward_scaling,
        "random_seed": random_seed,
    }
    
    try:
        train(
            start_date = TRAIN_START_DATE, 
            end_date = TRAIN_END_DATE,
            ticker_list = GP3_TICKERS, 
            data_source = 'alpaca',
            time_interval = '1D', 
            technical_indicator_list = INDICATORS,
            drl_lib ='elegantrl', 
            env = env,
            model_name = "ppo",
            if_vix = True,
            API_KEY = ALPACA_API_KEY, 
            API_SECRET = ALPACA_API_SECRET, 
            API_BASE_URL = API_BASE_URL,
            cwd = './gp3_testing', #current_working_dir
            break_step = 1e5,
            erl_params = erl_params,
        )

        episode_total_assets, max_drawdown, avg_reward = test(
            start_date = TEST_START_DATE, 
            end_date = TEST_END_DATE,
            ticker_list = GP3_TICKERS, 
            data_source = 'alpaca',
            time_interval = '1D', 
            technical_indicator_list = INDICATORS,
            drl_lib = 'elegantrl', 
            env = env, 
            model_name = "ppo",
            if_vix = True,
            API_KEY = ALPACA_API_KEY, 
            API_SECRET = ALPACA_API_SECRET, 
            API_BASE_URL = API_BASE_URL,
            cwd = './gp3_testing',
        )

        update_leaderboard('./gp3_testing', './gp3_leaderboard', episode_total_assets, max_drawdown, avg_reward, erl_params)
    
    except Exception as e:
        print(f'Error occurred: {e}')
        continue
    
    elapsed_time = time.time() - start_time
    if elapsed_time > run_time:
        break